In [1]:
import os
import shutil
import random
import pandas as pd
import sys
sys.path.append('../')
from src.data import download_images

## Get Data and sort it

In [2]:
# data = pd.read_csv('data/reviewed_data_5_13_24.csv') ##We can use this for training
data = pd.read_csv('../data/final_eval_data.csv')
# data = pd.read_csv('data/justin_dataset_may_models.csv').rename(columns={'new_url': 'download_url'}) ##This we can use to bolster the R rated dataset


In [4]:
data.groupby('label')['label'].count()

label
PG       7393
PG13    13425
R        5557
X        7622
XXX      6007
Name: label, dtype: int64

In [5]:
# Count the number of rows per label
label_counts = data.groupby('label')['label'].count()

# Get the minimum count among all labels
min_count = label_counts.min()

# Sample equal number of rows per label
balanced_data = data.groupby('label').apply(lambda x: x.sample(n=min_count, random_state=42)).reset_index(drop=True)

balanced_data.groupby('label')['label'].count()

/var/folders/lj/vxfv4wp1351_qdc939xvvn0m0000gn/T/ipykernel_6898/3553324668.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  balanced_data = data.groupby('label').apply(lambda x: x.sample(n=min_count, random_state=42)).reset_index(drop=True)


label
PG      5557
PG13    5557
R       5557
X       5557
XXX     5557
Name: label, dtype: int64

In [6]:
balanced_data.head()

,download_url,prompt,tags,label,datatype,mixtureModel
0,https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...,Generate a photograph of a realistic modern wo...,"beachwear, clothing, blonde, hair, person, adu...",PG,justins_eval,PG
1,https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...,"Gerudo Link, gerudolink, blonde, green see-thr...","man, blonde hair, blue eyes, crossdressing, de...",PG,justins_eval,PG13
2,https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...,photo of beautiful woman with perfect (red col...,"body freckles, nose, realistic, freckles, lips...",PG,training,NaN
3,https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...,"score_9, score_8_up, score_7_up, a red haired ...","woman, cartoon, head, face, person, publicatio...",PG,training,NaN
4,https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...,what do you want from me,"solo, strapless, realistic, parted lips, looki...",PG,training,NaN


In [7]:
data = {}

# Iterate over the DataFrame
for index, row in balanced_data.iterrows():
    data[index] = {
        "url": row['download_url'], 
        "prompt": row['prompt'],
        "tags": f"{row['tags']}",
        "label": row['label']
    }


In [8]:
output = './data/datasets/'
download_images(data, output)

6908228.jpg
4004989.jpg
6003423.jpg
4006158.jpg
8078722.jpg
9088482.jpg
3007056.jpg
8747511.jpg
4024102.jpg
8004788.jpg
8360905.jpg
6693408.jpg
3018039.jpg
10181032.jpg
5014836.jpg
7816759.jpg
10147314.jpg
5009480.jpg
4021462.jpg
5000504.jpg
10362226.jpg
Error processing image from https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7WA/98b7908b-02de-40a4-bee5-f62d990f83a3/width=450/10362226.jpg: write() argument must be str, not float
6912009.jpg
3512639.jpg
3020452.jpg5018655.jpg

5013974.jpg
10143356.jpg
4025150.jpg
8809016.jpg
5001636.jpg
10226350.jpg
7057597.jpg
7057267.jpg
Error processing image from https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7WA/bafee217-6c11-423b-8d8e-5f2b6b3dda53/width=450/7057267.jpg: write() argument must be str, not float
6013783.jpg
5007726.jpg
4014408.jpg
4003388.jpg
5001297.jpg
8143526.jpg
3014950.jpg
7057610.jpg
6716859.jpg
5012962.jpg
7057479.jpg
3016438.jpg
9000102.jpg
6693330.jpg
8748370.jpg
6261259.jpg
10243302.jpg
10244573.jpg
10259297.jpg
10244642.jpg


In [9]:
for i in os.listdir(output):
    path = os.path.join(output, i)
    print(i, f"imgs: {len([i for i in os.listdir(path) if i.endswith('.jpg')])}")
    print(i, f"text: {len([i for i in os.listdir(path) if i.endswith('.txt') and 'tags' not in i])}")
    print(i, f"tags: {len([i for i in os.listdir(path) if i.endswith('.txt') and 'tags' in i])}")


XXX imgs: 5455
XXX text: 5455
XXX tags: 5270
R imgs: 5477
R text: 5477
R tags: 5105
PG imgs: 5148
PG text: 5148
PG tags: 4756
PG13 imgs: 5311
PG13 text: 5311
PG13 tags: 4981
X imgs: 5461
X text: 5461
X tags: 5207


## Use loop for training data

In [10]:
# Define the source directory containing class-specific folders
source_dir = output

# Define the output directories for training and validation
output_train_dir = './data/datasets/may/train'
output_val_dir = './data/datasets/may/val'

# Create the output directories if they don't exist
os.makedirs(output_train_dir, exist_ok=True)
os.makedirs(output_val_dir, exist_ok=True)

# Iterate over each class-specific folder
for class_name in os.listdir(source_dir):
    if class_name not in ['PG', "PG13", "R", "X", "XXX"]:
        continue
    class_path = os.path.join(source_dir, class_name)
    
    if os.path.isdir(class_path):
        # List of all .txt files (excluding those with 'tags' in their name)
        files = [f.split('.')[0] for f in os.listdir(class_path) if f.endswith('.jpg')]
        
        num_files = len(files)
        train_split = int(num_files * 0.85)
        
        # Create lists of indices for training and validation sets
        shuffled_indices = list(range(num_files))
        random.shuffle(shuffled_indices)
        train_indices = shuffled_indices[:train_split]
        val_indices = shuffled_indices[train_split:]

        print(f"Class '{class_name}':")
        print(f"  Train indices: {len(train_indices)}")
        print(f"  Val indices: {len(val_indices)}")

        # Copy files into 'train' and 'val' directories
        for i in train_indices:
            txt_file = files[i] + '.txt'
            tag_file = files[i] + '_tags.txt'
            img_file = files[i] + '.jpg'  # Assume corresponding .jpg files
            
            # Source paths
            src_txt_path = os.path.join(class_path, txt_file)
            src_img_path = os.path.join(class_path, img_file)
            src_tag_path = os.path.join(class_path, tag_file)
            
            # Destination paths for training files
            dest_txt_path = os.path.join(output_train_dir, class_name, txt_file)
            dest_img_path = os.path.join(output_train_dir, class_name, img_file)
            dest_tag_path = os.path.join(output_train_dir, class_name, tag_file)
            
            # Create directories if they don't exist
            os.makedirs(os.path.join(output_train_dir, class_name), exist_ok=True)
            
            # Copy the files
            if os.path.exists(src_txt_path):
                shutil.copy2(src_txt_path, dest_txt_path)
            if os.path.exists(src_img_path):
                shutil.copy2(src_img_path, dest_img_path)
            if os.path.exists(src_tag_path):
                shutil.copy2(src_tag_path, dest_tag_path)

        for i in val_indices:
            txt_file = files[i] + '.txt'
            tag_file = files[i] + '_tags.txt'
            img_file = files[i] + '.jpg'  # Assume corresponding .jpg files
            
            # Source paths
            src_txt_path = os.path.join(class_path, txt_file)
            src_img_path = os.path.join(class_path, img_file)
            src_tag_path = os.path.join(class_path, tag_file)
            
            # Destination paths for validation files
            dest_txt_path = os.path.join(output_val_dir, class_name, txt_file)
            dest_img_path = os.path.join(output_val_dir, class_name, img_file)
            dest_tag_path = os.path.join(output_val_dir, class_name, tag_file)
            
            # Create directories if they don't exist
            os.makedirs(os.path.join(output_val_dir, class_name), exist_ok=True)
            
            # Copy the files
            if os.path.exists(src_txt_path):
                shutil.copy2(src_txt_path, dest_txt_path)
            if os.path.exists(src_img_path):
                shutil.copy2(src_img_path, dest_img_path)
            if os.path.exists(src_tag_path):
                shutil.copy2(src_tag_path, dest_tag_path)

print("File copying complete!")

Class 'XXX':
  Train indices: 4636
  Val indices: 819
Class 'R':
  Train indices: 4655
  Val indices: 822
Class 'PG':
  Train indices: 4375
  Val indices: 773
Class 'PG13':
  Train indices: 4514
  Val indices: 797
Class 'X':
  Train indices: 4641
  Val indices: 820
File copying complete!
